<a href="https://colab.research.google.com/github/berkekurt/langchain-tutorial/blob/main/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up the environment

In [1]:
!pip install -U langchain langchain-openai langchain-core python-dotenv openai
!pip install langchain-openai langchain-community
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: l

In [2]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# For normal accurate responses
# llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")


Enter your OpenAI API key: ··········


In [3]:
llm_model = "gpt-5.1"

In [4]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.0,
    model=llm_model,
)
result = chat.invoke("Hi")
print(result)

content='Hi! How can I help you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 7, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5.1-2025-11-13', 'system_fingerprint': None, 'id': 'chatcmpl-CciGqBmrINTrRau8cXJLvnQxS8mxk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--58831e19-c219-47a4-b5ca-bb953926588a-0' usage_metadata={'input_tokens': 7, 'output_tokens': 18, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# Ai Agent Example

In [5]:
import requests
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather in a given location"""
    response = requests.get(f"http://wttr.in/{location}?format=j1")
    return response.json()


agent = create_agent(
    model = "gpt-4.1-mini",
    tools = [get_weather],
    system_prompt = "You are a helpful weather assistant that can answer questions and help with tasks."
)
response = agent.invoke({"messages": [
    {"role": "user", "content": "What is the weather in Istanbul?"}
    ]
})
print(response["messages"][-1].content)
print(response["messages"][-1])


The current weather in Istanbul is partly cloudy with a temperature of about 15°C (59°F). The humidity is around 77%, and the wind is coming from the south-southwest at 12 km/h (7 mph). There is no precipitation at the moment.
content='The current weather in Istanbul is partly cloudy with a temperature of about 15°C (59°F). The humidity is around 77%, and the wind is coming from the south-southwest at 12 km/h (7 mph). There is no precipitation at the moment.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 9166, 'total_tokens': 9222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 9088}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CciH5ycRS3b6TqjQ2Mxv5KIIbj90C', 'service_tier': 'default', 'fi

# Complex Ai Agent Example

In [9]:
from dataclasses import dataclass

import requests
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

@dataclass
class Context:
    user_id: str

@dataclass
class ResponseFormat:
    summary: str
    temperature_celsius: float
    temperature_fahrenheit: float
    humidity: float


@tool("get_weather", description="Return weather information a given city", return_direct=False)
def get_weather(location: str):
    """Get the weather in a given location"""
    response = requests.get(f"http://wttr.in/{location}?format=j1")
    return response.json()

@tool("locate_user", description="Look up a user's city based on the context")
def locate_user(runtime: ToolRuntime[Context]):
    match runtime.context.user_id:
        case "ABC123":
            return "Istanbul"
        case "XYZ456":
            return "Ankara"
        case "XJKL222":
            return "Izmir"
        case _:
            return "Unknown"


model = init_chat_model("gpt-4.1-mini", temperature=0.3)


checkpointer = InMemorySaver()


agent = create_agent(
    model = model,
    tools = [get_weather, locate_user],
    system_prompt = "You are a helpful weather assistant that can answer questions and help with tasks.",
    context_schema = Context,
    response_format = ResponseFormat,
    checkpointer = checkpointer
)

config = {"configurable": {"thread_id": 1}}

response = agent.invoke({"messages": [
    {"role": "user", "content": "What is the weather like?"}
    ]},
    config = config,
    context = Context(user_id="ABC123")
)
print(response["structured_response"])
print(response["structured_response"].summary)
print(response["structured_response"].temperature_celsius)

ResponseFormat(summary='The current weather in Istanbul is partly cloudy with no precipitation. The temperature is around 15°C (59°F) with a humidity of 77%. Winds are coming from the south-southwest at about 12 km/h (7 mph).', temperature_celsius=15.0, temperature_fahrenheit=59.0, humidity=77.0)
The current weather in Istanbul is partly cloudy with no precipitation. The temperature is around 15°C (59°F) with a humidity of 77%. Winds are coming from the south-southwest at about 12 km/h (7 mph).
15.0


In [10]:
response = agent.invoke({"messages": [
    {"role": "user", "content": "What did I ask you?"}
    ]},
    config = config,
    context = Context(user_id="ABC123")
)
print(response["structured_response"])
print(response["structured_response"].summary)

ResponseFormat(summary='You asked about the current weather.', temperature_celsius=0.0, temperature_fahrenheit=0.0, humidity=0.0)
You asked about the current weather.
